# 02_selection_and_angles.ipynb

Objetivo: seleccionar muones, calcular ángulos entre pares de muones por evento y guardar un resumen por evento.

Autor: Benjamin Cabeza Durán (método) / ChatGPT (implementación)

In [ ]:
# Imports
import os, glob
import uproot
import awkward as ak
import numpy as np
import pandas as pd
from math import degrees

# Paths
DATA_DIR = 'data/raw'
OUT_DIR = 'results'
os.makedirs(OUT_DIR, exist_ok=True)

# Find sample
candidates = sorted(glob.glob(os.path.join(DATA_DIR, '*.root')) + glob.glob(os.path.join(DATA_DIR, '*.root.gz')))
if not candidates:
    raise SystemExit('No sample ROOT file found in data/raw/. Añade un .root o un enlace simbólico.')
SAMPLE = candidates[0]
print('Using sample:', SAMPLE)

# Open file and select tree
f = uproot.open(SAMPLE)
# choose Events tree heuristically
tree_name = None
for k in f.keys():
    if 'Events' in k:
        tree_name = k
        break
if tree_name is None:
    tree_name = list(f.keys())[0]
print('Tree chosen:', tree_name)
tr = f[tree_name]

In [ ]:
# Determine available muon branches (common NanoAOD names)
branches = list(tr.keys())
print('Number of branches in tree:', len(branches))

# Candidates for muon kinematics
pt_names = [b for b in branches if 'Muon' in b and 'pt' in b.lower()]
eta_names = [b for b in branches if 'Muon' in b and 'eta' in b.lower()]
phi_names = [b for b in branches if 'Muon' in b and 'phi' in b.lower()]

print('pt candidates:', pt_names[:5])
print('eta candidates:', eta_names[:5])
print('phi candidates:', phi_names[:5])

# pick the likely canonical names if present
pt_branch = 'Muon_pt' if 'Muon_pt' in branches else (pt_names[0] if pt_names else None)
eta_branch = 'Muon_eta' if 'Muon_eta' in branches else (eta_names[0] if eta_names else None)
phi_branch = 'Muon_phi' if 'Muon_phi' in branches else (phi_names[0] if phi_names else None)

if not (pt_branch and eta_branch and phi_branch):
    raise SystemExit('No se encontraron las ramas Muon_pt / Muon_eta / Muon_phi. Revisa las ramas impresas arriba.')

print('Using branches:', pt_branch, eta_branch, phi_branch)

In [ ]:
# Read arrays (use awkward)
# Limit to a subset of entries if dataset is huge (opcional): entry_stop = 200000 -> None reads all
entry_stop = None
mu_pt = tr[pt_branch].array(entry_stop=entry_stop, library='ak')
mu_eta = tr[eta_branch].array(entry_stop=entry_stop, library='ak')
mu_phi = tr[phi_branch].array(entry_stop=entry_stop, library='ak')

# number of muons per event
n_mu = ak.num(mu_pt)
print('Events read (approx):', len(mu_pt))
print('Muon counts stats: min, median, max ->', int(ak.min(n_mu)), int(ak.median(n_mu)), int(ak.max(n_mu)))

In [ ]:
# Compute 3D momentum components from (pt, eta, phi)
# pz = pt * sinh(eta)
px = mu_pt * np.cos(mu_phi)
py = mu_pt * np.sin(mu_phi)
pz = mu_pt * np.sinh(mu_eta)

# Build per-event vector arrays
pvec = ak.zip({'px': px, 'py': py, 'pz': pz, 'pt': mu_pt})

# For events with at least 2 muons, compute pairwise combinations
has_pairs = n_mu >= 2
print('Events with >=2 muons:', int(ak.sum(has_pairs)))

# combinations per event (pairs): returns arrays of shape (events, n_pairs, 2)
pairs = ak.combinations(pvec, 2, axis=1)
# p0 and p1
p0 = pairs['0']
p1 = pairs['1']

# dot product and norms
dot = p0['px']*p1['px'] + p0['py']*p1['py'] + p0['pz']*p1['pz']
norm0 = np.sqrt(p0['px']**2 + p0['py']**2 + p0['pz']**2)
norm1 = np.sqrt(p1['px']**2 + p1['py']**2 + p1['pz']**2)

# cosine and angle
cosang = dot / (norm0 * norm1)
# numerical safety
cosang = ak.where(ak.is_none(cosang), 0.0, cosang)
cosang = ak.clip(cosang, -1.0, 1.0)
angle_rad = np.arccos(cosang)
angle_deg = angle_rad * 180.0 / np.pi

# per-event summaries
n_pairs = ak.num(angle_rad)
min_angle = ak.min(angle_deg, axis=1)
mean_angle = ak.mean(angle_deg, axis=1)
max_angle = ak.max(angle_deg, axis=1)

# If some events have no pairs, ensure placeholders
min_angle = ak.where(n_pairs>0, min_angle, np.nan)
mean_angle = ak.where(n_pairs>0, mean_angle, np.nan)
max_angle = ak.where(n_pairs>0, max_angle, np.nan)

print('Computed pair-angle summaries.')

In [ ]:
# Prepare event identifiers if available (run, luminosityBlock, event)
run = tr['run'].array(entry_stop=entry_stop, library='ak') if 'run' in branches else ak.arange(len(mu_pt))
lumib = tr['luminosityBlock'].array(entry_stop=entry_stop, library='ak') if 'luminosityBlock' in branches else ak.zeros_like(run)
evt = tr['event'].array(entry_stop=entry_stop, library='ak') if 'event' in branches else ak.arange(len(mu_pt))

# Build pandas DataFrame summary (flatten per-event values)
summary = pd.DataFrame({
    'run': ak.to_numpy(run),
    'luminosityBlock': ak.to_numpy(lumib),
    'event': ak.to_numpy(evt),
    'n_mu': ak.to_numpy(n_mu),
    'n_pairs': ak.to_numpy(n_pairs),
    'min_angle_deg': ak.to_numpy(min_angle),
    'mean_angle_deg': ak.to_numpy(mean_angle),
    'max_angle_deg': ak.to_numpy(max_angle),
})

out_csv = os.path.join(OUT_DIR, 'angles_summary.csv')
summary.to_csv(out_csv, index=False)
print('Wrote summary to', out_csv)

# Quick stats
print(summary.describe(include='all'))

# Save a small sample for inspection
out_html = os.path.join(OUT_DIR, 'angles_summary_head.html')
summary.head(200).to_html(out_html, index=False)
print('Saved head table to', out_html)